In [ ]:
# Standard library imports
import subprocess
import sys
from pathlib import Path

from multiview_mapping_toolkit.utils.visualization import show_segmentation_labels

# Imports from the constants
sys.path.append("../..")
from constants import (
    MMSEG_PYTHON,
    MMSEG_UTILS_PYTHON,
    INFERENCE_SCRIPT,
    VIS_PREDS_SCRIPT,
    get_inference_image_folder,
    get_work_dir,
    get_prediction_folder,
)

# Set constants
You should be able to define most of the behavior from these constants

In [ ]:
TEST_SITE = "none"
TRAINING_SITES = ["none", "none"]

# Generate predictions with the model

In [ ]:
WORK_DIR = get_work_dir(training_sites=TRAINING_SITES)
INFERENCE_IMAGE_FOLDER = get_inference_image_folder(site_name=TEST_SITE)
PREDICTIONS_FOLDER = get_prediction_folder(
    prediction_site=TEST_SITE, training_sites=TRAINING_SITES
)


config_file = list(Path(WORK_DIR).glob("*py"))[0]
checkpoint_file = Path(WORK_DIR, "iter_10000.pth")
BATCH_SIZE = 2

subprocess.run(
    f"{MMSEG_PYTHON} {INFERENCE_SCRIPT} {config_file} "
    + f"{checkpoint_file} {INFERENCE_IMAGE_FOLDER} {PREDICTIONS_FOLDER} --batch-size {BATCH_SIZE}",
    shell=True,
)

# Visualize predictions

In [ ]:
# Add _vis to the folder name. Can't just add _vis to the str representation because it might have a trailing slash
pred_vis_dir = Path(
    Path(PREDICTIONS_FOLDER).parent, Path(PREDICTIONS_FOLDER).parts[-1] + "_vis"
)
STRIDE = 10

subprocess.run(
    f"{MMSEG_UTILS_PYTHON} {VIS_PREDS_SCRIPT} --image-dir {INFERENCE_IMAGE_FOLDER} "
    + f"--seg-dir {PREDICTIONS_FOLDER} --output-dir {pred_vis_dir} --stride {STRIDE}",
    shell=True,
)

In [ ]:
show_segmentation_labels(
    label_folder=PREDICTIONS_FOLDER, image_folder=INFERENCE_IMAGE_FOLDER
)